In [1]:
import os
import pickle
import tqdm
import json


In [2]:
section_name_set = ['admission, date:', 'service:', 'allergies:','chief, complaint:','major, surgical, or, invasive, procedure:','history, of, present, illness:', 'past, medical, history:', 'past, surgical, history:', 'allergies:', 'medications, on, admission:', 'physical, examination:', 'physical, exam:', 'hospital, course:', 'discharge, diagnosis:', 'discharge, medications:', 'discharge, instructions:', 'follow-up, instructions:', 'follow-up, plan:', 'follow-up, medications:', 'follow-up, diagnosis:',  'medications:', 'initial, laboratory, studies:', 'partnent, results:', 'family, history:', 'social, history:']
                    
folder = 'word_label_pairs'

# list of all path
all_word_label_pairs_list = [os.path.join(folder, file) for file in os.listdir(folder)]

all_section_line_id = {}

for all_word_label_pairs_path in tqdm.tqdm(all_word_label_pairs_list, total=len(all_word_label_pairs_list)):
    # read pickle
    with open(all_word_label_pairs_path, 'rb') as f:
        word_label_pairs = pickle.load(f)
    row_id = all_word_label_pairs_path.replace('.pkl', '').replace('word_label_pairs/', '')
    word_list = [i[0] for key, i in word_label_pairs[0].items()]
    all_ent_str = ", ".join(word_list)
    all_ent_lines = all_ent_str.split('\n')

    section_line_id = {}
    for i, line in enumerate(all_ent_lines):
        # if line include any of section_name_set
        if any([section_name in line.lower() for section_name in section_name_set]):
            # get section name
            section_name = [section_name for section_name in section_name_set if section_name in line.lower()][0]
            section_line_id[section_name] = i
    all_section_line_id[row_id] = {"section_line_id":section_line_id}


100%|██████████| 59652/59652 [02:24<00:00, 413.42it/s] 


In [3]:
json_path = '../NEextraction/entity_dict_negated.json'
json_list = []
with open(json_path, 'r') as f:
    # read lines
    lines = f.readlines()
    # each line is a json object
    for line in lines:
        json_list.append(json.loads(line))

In [4]:
new_json_list = []
for row in json_list:
    row_id = row['row_id']
    entities = row['entities']
    section_name_dict = all_section_line_id[row_id]['section_line_id']
    break_line_indices = [i for i, entity in enumerate(entities) if entity == '<br>']
    #ent_list = ' '.join(all_section_line_id[row_id]['ent']).split('\n')
    new_section_name_dict = {}  
    for k,v in section_name_dict.items():
        if v == 0:
            new_section_name_dict[0] = k
        else:
            new_section_name_dict[break_line_indices[v-1]+1] = k
    new_entities = []
    for i, entity in enumerate(entities):
        if i in new_section_name_dict.keys():
            new_entities.append("~"+" ".join(new_section_name_dict[i].split(','))+"~")
            new_entities.append(entity)
        else:
            new_entities.append(entity)
    new_json_list.append({"row_id": row_id, "entities": new_entities})
    
     

In [5]:
save_path  = '../NEextraction/sectioned_entity_dict_negated.json'

# save as lines of json
with open(save_path, 'w') as f:
    for row in new_json_list:
        f.write(json.dumps(row) + '\n')

# read

#json_#path = '../NEextraction/sectioned_entity_dict_negated.json'
#json_list = []
#with open(json_path, 'r') as f:
#    # read lines
#    lines = f.readlines()
#    # each line is a json object
#    for line in lines:
#        json_list.append(json.loads(line))

In [ ]:
comman_removed_section_name_set = []
for section_name in section_name_set:
    comman_removed_section_name_set.append(section_name.replace(',', ' '))
print(comman_removed_section_name_set)